# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

import os
import yaml
from crewai import Agent, Task, Crew

In [2]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

In [3]:
import pandas as pd
import numpy as np

# Read the CSV
df = pd.read_csv("restaurant_expenses.csv")

print("🔍 Initial data check:")
print(f"Rows: {len(df)}")
print(f"Columns: {df.columns.tolist()}")

# Clean Amount column
df['Amount'] = df['Amount'].replace('[\$,]', '', regex=True).astype(float)

# Clean Category column
df['Category'] = df['Category'].str.strip().str.lower().replace('resturants', 'restaurants')

# Convert Date to datetime
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Clean Description column (remove special characters that might cause issues)
df['Description'] = df['Description'].str.replace('?', "'", regex=False)  # Fix AMMA?S -> AMMA'S
df['Description'] = df['Description'].str.strip()

# Check for any null values
print(f"\n🔍 Null values check:")
print(df.isnull().sum())

# Remove any rows with null amounts or dates
initial_rows = len(df)
df = df.dropna(subset=['Amount', 'Date'])
if len(df) < initial_rows:
    print(f"\n⚠️ Removed {initial_rows - len(df)} rows with null values")

# Sort by date
df = df.sort_values('Date')

# Summary statistics
print(f"\n📊 Summary Statistics:")
print(f"Total amount: ${df['Amount'].sum():.2f}")
print(f"Number of transactions: {len(df)}")
print(f"Average transaction: ${df['Amount'].mean():.2f}")
print(f"Date range: {df['Date'].min().date()} to {df['Date'].max().date()}")

# Top restaurants
print(f"\n🏪 Top 10 Restaurants by Total Spending:")
top_restaurants = df.groupby('Description')['Amount'].agg(['count', 'sum', 'mean']).round(2)
top_restaurants.columns = ['Visits', 'Total', 'Average']
top_restaurants = top_restaurants.sort_values('Total', ascending=False)
print(top_restaurants.head(10))

# Check the expected total
expected_total = 951.67
actual_total = df['Amount'].sum()
difference = abs(expected_total - actual_total)

if difference > 0.01:
    print(f"\n⚠️ Warning: Total (${actual_total:.2f}) differs from expected (${expected_total:.2f}) by ${difference:.2f}")
else:
    print(f"\n✅ Total matches expected amount: ${actual_total:.2f}")

# Save cleaned data
df.to_csv("restaurant_expenses_cleaned.csv", index=False)
print(f"\n💾 Saved cleaned data to: restaurant_expenses_cleaned.csv")

# Additional validation - check for potential duplicates
print(f"\n🔍 Checking for potential duplicates:")
potential_dupes = df[df.duplicated(subset=['Date', 'Amount', 'Description'], keep=False)]
if len(potential_dupes) > 0:
    print(f"Found {len(potential_dupes)} potential duplicate entries:")
    print(potential_dupes[['Date', 'Amount', 'Description']].head(10))
else:
    print("No potential duplicates found")

# Create a LangChain-friendly version (no special characters)
df_langchain = df.copy()
df_langchain['Description'] = df_langchain['Description'].str.replace('[^a-zA-Z0-9\s]', ' ', regex=True)
df_langchain['Description'] = df_langchain['Description'].str.strip().str.replace('\s+', ' ', regex=True)
df_langchain.to_csv("restaurant_expenses_langchain.csv", index=False)
print(f"\n💾 Saved LangChain-friendly version to: restaurant_expenses_langchain.csv")

🔍 Initial data check:
Rows: 61
Columns: ['Date', 'Amount', 'Description', 'Category']

🔍 Null values check:
Date           0
Amount         0
Description    0
Category       0
dtype: int64

📊 Summary Statistics:
Total amount: $932.02
Number of transactions: 61
Average transaction: $15.28
Date range: 2025-07-01 to 2025-07-31

🏪 Top 10 Restaurants by Total Spending:
                     Visits   Total  Average
Description                                 
CHIPOTLE 1233             7  116.91    16.70
Aramark                  23   97.93     4.26
AMMA'S KITCHEN            2   89.26    44.63
CHARM THAI CUISINE        1   88.47    88.47
CHEESECAKE                1   86.55    86.55
THAI NAM PHET             1   85.02    85.02
SUMMIT THAI               1   60.13    60.13
Uber Eats                 2   54.69    27.34
STARBUCKS                 4   35.89     8.97
EAST GATE VERONA NJ       1   29.72    29.72

⚠️ Warning: Total ($932.02) differs from expected ($951.67) by $19.65

💾 Saved cleaned data 

In [4]:
from crewai_tools import FileReadTool
csv_tool = FileReadTool(file_path='./restaurant_expenses_langchain.csv')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pydantic/_internal/_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)


In [ ]:
print(csv_tool.run())


In [ ]:
import pandas as pd
from io import StringIO

csv_raw = csv_tool.run()
df = pd.read_csv(StringIO(csv_raw))

print(df.head())
print(f"Total Transactions: {len(df)}")
print(f"Total Amount Spent: ${df['Amount'].sum():.2f}")


In [5]:
# Creating Agents
budget_reader_agent = Agent(
  config=agents_config['budget_reader_agent'],
  tools=[csv_tool]
)

expense_tracker_agent = Agent(
  config=agents_config['expense_tracker_agent'],
  tools=[csv_tool]
)


burn_rate_calculator_agent = Agent(
  config=agents_config['burn_rate_calculator_agent'],
  allow_code_execution=False
)

visualization_agent = Agent(
  config=agents_config['visualization_agent'],
  allow_code_execution=False
)

alert_agents = Agent(
  config=agents_config['alert_agents'],
  allow_code_execution=False
)

reporting_agent = Agent(
  config=agents_config['reporting_agent'],
  allow_code_execution=False
)

# Creating Tasks
budget_reader = Task(
  config=tasks_config['budget_reader'],
  agent=budget_reader_agent
)

expense_tracker = Task(
  config=tasks_config['expense_tracker'],
  agent=expense_tracker_agent
)

burner_rate_calculator = Task(
  config=tasks_config['burner_rate_calculator'],
  agent=burn_rate_calculator_agent
)

visualization_assembly = Task(
  config=tasks_config['visualization_assembly'],
  agent=visualization_agent
)

alert_assembly = Task(
  config=tasks_config['alert_assembly'],
  agent=alert_agents,
    context=[budget_reader, expense_tracker, burner_rate_calculator]
)

final_report_assembly = Task(
    config=tasks_config['final_report_assembly'],
    agent=reporting_agent,  # This is the agent that assembles the report
    context=[
        budget_reader, 
        expense_tracker, 
        burner_rate_calculator,
        visualization_assembly,
        alert_assembly
        ]
)


# Creating Crew
support_report_crew = Crew(
  agents=[
    budget_reader_agent,
    expense_tracker_agent,
    burn_rate_calculator_agent,
    visualization_agent,
    alert_agents,
    reporting_agent
  ],
  tasks=[
    budget_reader,
    expense_tracker,
    burner_rate_calculator,
    visualization_assembly,
    alert_assembly,
    final_report_assembly
  ],
  verbose=True
)


In [ ]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    model_name="gpt-4o",
    temperature=0.9,
    your_key=os.getenv(""),
    verbose=True
)

/var/folders/k3/7fd1bsr57z39k2d0tc98r85h0000gn/T/ipykernel_54809/1838640164.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [ ]:
from langchain.callbacks import get_openai_callback
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o")

with get_openai_callback() as cb:
    llm.invoke("Explain the moon landing")
    print(cb)

In [7]:
output = support_report_crew.kickoff(inputs={
  "monthly_budget": 500,
  "expenses_csv": "restaurant_expenses_langchain.csv"
})


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 62ac161b-f038-4fd9-b287-26752b8ec63b                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Budget Reader Engine                                                                                    │
│                                                                                                                 │
│  Task: Load and interpret pre-allocated budget data for the expense tracking system.                            │
│  Behavior: - If `monthly_budget` is provided as input, use that as the total budget. - Otherwise, fall back to  │
│  the default: $500.                                                                                             │
│  - Allocate 100% of the monthly budget to the "restaurants" category.                                           │
│  Responsibilities: - Parse budget structure and allocation breakdown. - Validate that the total allocation      │
│  equals the budget. - Compute daily and weekly limits for reference. - Share structured budget context with     │
│  other agents.                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Budget Reader Engine                                                                                    │
│                                                                                                                 │
│  Thought: I need to load and interpret the pre-allocated budget data for the expense tracking system. The       │
│  first step is to read the content of the budget file to gather the necessary information.                      │
│                                                                                                                 │
│  Using Tool: Read a file's content                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{}"                                                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Date,Amount,Description,Category                                                                               │
│  2025-07-01,10.5,CHIPOTLE 1233,restaurants                                                                      │
│  2025-07-01,14.8,Aramark,restaurants                                                                            │
│  2025-07-01,2.44,Aramark,restaurants                                                                            │
│  2025-07-02,9.47,WENDYS 6702,restaurants                                                                        │
│  2025-07-02,14.18,CHIPOTLE 1233,restaurants                                                                     │
│  2025-07-02,5.77,Aramark,restaurants                                                                            │
│  2025-07-02,2.44,Aramark,restaurants                                                                            │
│  2025-07-02,8.36,Aramark,restaurants                                                                            │
│  2025-07-02,7.35,Aramark,restaurants                                                                            │
│  2025-07-04,7.96,DUNKIN,restaurants                                                                             │
│  2025-07-05,3.33,BURGER KING 10618,restaurants                                                                  │
│  2025-07-05,8.77,BURGER KING 10618,restaurants                                                                  │
│  2025-07-06,11.7,LAST LICKS EAST,restaurants                                                                    │
│  2025-07-06,11.73,GANESH S DOSA,restaurants                                                                     │
│  2025-07-06,5.33,GANESH S DOSA,restaurants                                                                      │
│  2025-07-07,3.12,Aramark,restaurants                                                                            │
│  2025-07-07,3.72,Aramark,restaurants                                                                            │
│  2025-07-07,5.94,Aramark,restaurants                                                                            │
│  2025-07-07,2.44,Aramark,restaurants                                                                            │
│  2025-07-07,7.45,SMILEYS FRESH PROD,restaurants                                                                 │
│  2025-07-08,5.53,Aramark,restaurants                                                                            │
│  2025-07-08,2.24,Aramark,restaurants                                                                            │
│  2025-07-08,8.19,DUNKIN 331892,restaurants                                                                      │
│  2025-07-08,11.62,STARBUCKS STORE 56775,restaurants                                                             │
│  2025-07-08,14.18,CHIPOTLE 1233,restaurants                                                                     │
│  2025-07-09,2.12,Aramark,restaurants                                                                            │
│  2025-07-09,9.79,Aramark,restaurants                                                                            │
│  2025-07-10,85.02,THAI NAM PHET,restaurants                                                                     │
│  2025-07-12,6.02,STARBUCKS,restaurants                                                                          │
│  2025-07-12,29.72,EAST GATE VERONA NJ,restaurants                                                               │
│  2025-07-12,17.77,COZY TEA LOFT B,restaurants                                                                   │
│  2025-07-12,6.68,DAIRY QUEEN,restaurants              

...

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: c945f807-d9a9-4b3c-bd68-f404a68c69a9                                                                     │
│  Agent: Alert User Engine                                                                                       │
│                                                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Final Report Engine                                                                                     │
│                                                                                                                 │
│  Task: Compile a Markdown report including:                                                                     │
│  - Budget summary - Burn rate chart - Alert logs - Daily spending table - Actionable insights                   │
│  Responsibilities: - Pull data only from upstream outputs (no recalculations) - Present information clearly     │
│  using charts, tables, and headlines                                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Final Report Engine                                                                                     │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```html                                                                                                        │
│  <!DOCTYPE html>                                                                                                │
│  <html lang="en">                                                                                               │
│  <head>                                                                                                         │
│      <meta charset="UTF-8">                                                                                     │
│      <meta name="viewport" content="width=device-width, initial-scale=1.0">                                     │
│      <title>Budget Report</title>                                                                               │
│      <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>                                           │
│  </head>                                                                                                        │
│  <body>                                                                                                         │
│      <h1>Budget Report</h1>                                                                                     │
│                                                                                                                 │
│      <h2>1. Budget Summary</h2>                                                                                 │
│      <p><strong>Total Budget:</strong> $500</p>                                                                 │
│      <p><strong>Total Spent:</strong> $932.02</p>                                                               │
│      <p><strong>Over Budget:</strong> $432.02</p>                                                               │
│                                                                                                                 │
│      <h2>2. Burn Rate</h2>                                                                                      │
│      <p><strong>Daily Burn Rate:</strong> $29.93</p>                                                            │
│      <p><strong>Projected Exhaustion Date:</strong> 2025-07-17</p>                                              │
│                                                                                                                 │
│      <h2>3. Alerts</h2>                                                                                         │
│      <ul>                                                                                                       │
│          <li>Crossed 50% budget on 2025-07-10</li>                                                              │
│          <li>Crossed 75% on 2025-07-15</li>                                                                     │
│          <li>Crossed 90% on 2025-07-17</li>                                                                     │
│          <li>Projected to exhaust budget by 2025-07-17</li>                                                     │
│      </ul>                                                                                                      │
│                                                                                                                 │
│      <h2>4. Burn Rate Chart</h2>                       

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: bf450c53-0487-49bb-b5ab-9ccf1914c619                                                                     │
│  Agent: Final Report Engine                                                                                     │
│                                                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 62ac161b-f038-4fd9-b287-26752b8ec63b                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```html                                                                                          │
│  <!DOCTYPE html>                                                                                                │
│  <html lang="en">                                                                                               │
│  <head>                                                                                                         │
│      <meta charset="UTF-8">                                                                                     │
│      <meta name="viewport" content="width=device-width, initial-scale=1.0">                                     │
│      <title>Budget Report</title>                                                                               │
│      <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>                                           │
│  </head>                                                                                                        │
│  <body>                                                                                                         │
│      <h1>Budget Report</h1>                                                                                     │
│                                                                                                                 │
│      <h2>1. Budget Summary</h2>                                                                                 │
│      <p><strong>Total Budget:</strong> $500</p>                                                                 │
│      <p><strong>Total Spent:</strong> $932.02</p>                                                               │
│      <p><strong>Over Budget:</strong> $432.02</p>                                                               │
│                                                                                                                 │
│      <h2>2. Burn Rate</h2>                                                                                      │
│      <p><strong>Daily Burn Rate:</strong> $29.93</p>                                                            │
│      <p><strong>Projected Exhaustion Date:</strong> 2025-07-17</p>                                              │
│                                                                                                                 │
│      <h2>3. Alerts</h2>                                                                                         │
│      <ul>                                                                                                       │
│          <li>Crossed 50% budget on 2025-07-10</li>                                                              │
│          <li>Crossed 75% on 2025-07-15</li>                                                                     │
│          <li>Crossed 90% on 2025-07-17</li>                                                                     │
│          <li>Projected to exhaust budget by 2025-07-17</li>                                                     │
│      </ul>                                                                                                      │
│                                                       

In [8]:
from IPython.display import display, Markdown
display(Markdown(output.raw))


```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Budget Report</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
</head>
<body>
    <h1>Budget Report</h1>

    <h2>1. Budget Summary</h2>
    <p><strong>Total Budget:</strong> $500</p>
    <p><strong>Total Spent:</strong> $932.02</p>
    <p><strong>Over Budget:</strong> $432.02</p>

    <h2>2. Burn Rate</h2>
    <p><strong>Daily Burn Rate:</strong> $29.93</p>
    <p><strong>Projected Exhaustion Date:</strong> 2025-07-17</p>

    <h2>3. Alerts</h2>
    <ul>
        <li>Crossed 50% budget on 2025-07-10</li>
        <li>Crossed 75% on 2025-07-15</li>
        <li>Crossed 90% on 2025-07-17</li>
        <li>Projected to exhaust budget by 2025-07-17</li>
    </ul>

    <h2>4. Burn Rate Chart</h2>
    <div id="burn-rate-chart"></div>
    
    <script>
        const cumulativeSpend = {
            "2025-07-01": 27.74,
            "2025-07-02": 79.25,
            "2025-07-03": 79.25,
            "2025-07-04": 87.21,
            "2025-07-05": 99.31,
            "2025-07-06": 127.04,
            "2025-07-07": 138.50,
            "2025-07-08": 175.82,
            "2025-07-09": 187.73,
            "2025-07-10": 272.75,
            "2025-07-11": 272.75,
            "2025-07-12": 285.17,
            "2025-07-13": 327.80,
            "2025-07-14": 330.24,
            "2025-07-15": 338.26,
            "2025-07-16": 343.14,
            "2025-07-17": 408.73,
            "2025-07-18": 419.26,
            "2025-07-19": 440.27,
            "2025-07-20": 486.90,
            "2025-07-21": 489.34,
            "2025-07-22": 489.34,
            "2025-07-23": 582.25,
            "2025-07-24": 621.05,
            "2025-07-25": 621.05,
            "2025-07-26": 634.57,
            "2025-07-27": 721.12,
            "2025-07-28": 754.95,
            "2025-07-29": 757.39,
            "2025-07-30": 778.72,
            "2025-07-31": 932.02
        };

        const dates = Object.keys(cumulativeSpend);
        const spends = Object.values(cumulativeSpend);
        const budget = 500;

        const trace1 = {
            x: dates,
            y: spends,
            mode: 'lines+markers',
            name: 'Cumulative Spend',
            line: {color: 'blue'}
        };

        const trace2 = {
            x: dates,
            y: Array(dates.length).fill(budget),
            mode: 'lines',
            name: 'Budget',
            line: {color: 'red'}
        };

        const data = [trace1, trace2];

        const layout = {
            title: 'Burn Rate Chart',
            xaxis: {
                title: 'Date'
            },
            yaxis: {
                title: 'Dollar Amount',
                rangemode: 'tozero'
            },
            shapes: [
                { type: 'line', x0: '2025-07-10', x1: '2025-07-10', y0: 0, y1: budget, line: {color: 'green', width: 2, dash: 'dash'} },
                { type: 'line', x0: '2025-07-15', x1: '2025-07-15', y0: 0, y1: budget, line: {color: 'orange', width: 2, dash: 'dash'} },
                { type: 'line', x0: '2025-07-17', x1: '2025-07-17', y0: 0, y1: budget, line: {color: 'purple', width: 2, dash: 'dash'} }
            ],
            annotations: [
                { x: '2025-07-10', y: budget + 50, xref: 'x', yref: 'y', text: 'Crossed 50%', showarrow: true, arrowhead: 2 },
                { x: '2025-07-15', y: budget + 50, xref: 'x', yref: 'y', text: 'Crossed 75%', showarrow: true, arrowhead: 2 },
                { x: '2025-07-17', y: budget + 50, xref: 'x', yref: 'y', text: 'Crossed 90%', showarrow: true, arrowhead: 2 }
            ]
        };

        Plotly.newPlot('burn-rate-chart', data, layout);
    </script>

    <h2>5. Daily Spending Table</h2>
    <table>
        <thead>
            <tr><th>Date</th><th>Amount</th><th>Cumulative</th></tr>
        </thead>
        <tbody>
            <tr><td>2025-07-01</td><td>$10.50</td><td>$27.74</td></tr>
            <tr><td>2025-07-02</td><td>$14.18</td><td>$79.25</td></tr>
            <tr><td>2025-07-03</td><td>$0.00</td><td>$79.25</td></tr>
            <tr><td>2025-07-04</td><td>$7.96</td><td>$87.21</td></tr>
            <tr><td>2025-07-05</td><td>$8.77</td><td>$99.31</td></tr>
            <tr><td>2025-07-06</td><td>$11.73</td><td>$127.04</td></tr>
            <tr><td>2025-07-07</td><td>$13.22</td><td>$138.50</td></tr>
            <tr><td>2025-07-08</td><td>$17.54</td><td>$175.82</td></tr>
            <tr><td>2025-07-09</td><td>$9.79</td><td>$187.73</td></tr>
            <tr><td>2025-07-10</td><td>$85.02</td><td>$272.75</td></tr>
            <tr><td>2025-07-11</td><td>$0.00</td><td>$272.75</td></tr>
            <tr><td>2025-07-12</td><td>$29.72</td><td>$285.17</td></tr>
            <tr><td>2025-07-13</td><td>$42.63</td><td>$327.80</td></tr>
            <tr><td>2025-07-14</td><td>$2.44</td><td>$330.24</td></tr>
            <tr><td>2025-07-15</td><td>$6.02</td><td>$338.26</td></tr>
            <tr><td>2025-07-16</td><td>$4.88</td><td>$343.14</td></tr>
            <tr><td>2025-07-17</td><td>$65.59</td><td>$408.73</td></tr>
            <tr><td>2025-07-18</td><td>$10.53</td><td>$419.26</td></tr>
            <tr><td>2025-07-19</td><td>$21.01</td><td>$440.27</td></tr>
            <tr><td>2025-07-20</td><td>$46.63</td><td>$486.90</td></tr>
            <tr><td>2025-07-21</td><td>$9.46</td><td>$489.34</td></tr>
            <tr><td>2025-07-22</td><td>$0.00</td><td>$489.34</td></tr>
            <tr><td>2025-07-23</td><td>$88.47</td><td>$582.25</td></tr>
            <tr><td>2025-07-24</td><td>$39.56</td><td>$621.05</td></tr>
            <tr><td>2025-07-25</td><td>$0.00</td><td>$621.05</td></tr>
            <tr><td>2025-07-26</td><td>$13.52</td><td>$634.57</td></tr>
            <tr><td>2025-07-27</td><td>$86.55</td><td>$721.12</td></tr>
            <tr><td>2025-07-28</td><td>$34.99</td><td>$754.95</td></tr>
            <tr><td>2025-07-29</td><td>$2.44</td><td>$757.39</td></tr>
            <tr><td>2025-07-30</td><td>$21.33</td><td>$778.72</td></tr>
            <tr><td>2025-07-31</td><td>$84.38</td><td>$932.02</td></tr>
        </tbody>
    </table>
</body>
</html>
```

This HTML report includes a comprehensive analysis of the budget expenditures, a burn rate chart, and a detailed daily spending table for stakeholder review. يُ

In [11]:
import markdown
html_string = markdown.markdown(output.raw, extensions=['tables'])
display(HTML(html_string))

Date,Amount,Cumulative
2025-07-01,$10.50,$27.74
2025-07-02,$14.18,$79.25
2025-07-03,$0.00,$79.25
2025-07-04,$7.96,$87.21
2025-07-05,$8.77,$99.31
2025-07-06,$11.73,$127.04
2025-07-07,$13.22,$138.50
2025-07-08,$17.54,$175.82
2025-07-09,$9.79,$187.73
2025-07-10,$85.02,$272.75


In [10]:
from IPython.core.display import display, HTML
display(HTML(output.raw))


/var/folders/k3/7fd1bsr57z39k2d0tc98r85h0000gn/T/ipykernel_54809/2892000134.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


Date,Amount,Cumulative
2025-07-01,$10.50,$27.74
2025-07-02,$14.18,$79.25
2025-07-03,$0.00,$79.25
2025-07-04,$7.96,$87.21
2025-07-05,$8.77,$99.31
2025-07-06,$11.73,$127.04
2025-07-07,$13.22,$138.50
2025-07-08,$17.54,$175.82
2025-07-09,$9.79,$187.73
2025-07-10,$85.02,$272.75


In [12]:
with open("budget_report.html", "w") as f:
    f.write(output.raw)

In [ ]:
import matplotlib.pyplot as plt

# ... (prepare the matplotlib chart like before)

plt.savefig("burn_chart.png")

# Then use HTML to embed:
html = """
<html>
<head><title>Burn Rate Chart</title></head>
<body>
<h2>Burn Rate Chart - July 2025</h2>
<img src="burn_chart.png" alt="Burn Chart" width="800">
</body>
</html>
"""

with open("burn_chart.html", "w") as f:
    f.write(html)


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import io
import base64

def create_burn_chart(expenses_df, monthly_limit):
    expenses_df['Date'] = pd.to_datetime(expenses_df['Date'])
    expenses_df = expenses_df.sort_values('Date')
    expenses_df['Cumulative Spend'] = expenses_df['Amount'].cumsum()

    plt.figure(figsize=(10, 6))
    plt.plot(expenses_df['Date'], expenses_df['Cumulative Spend'], label='Cumulative Spend')
    plt.axhline(y=monthly_limit, color='r', linestyle='--', label='Monthly Limit (${})'.format(monthly_limit))
    plt.xlabel("Date")
    plt.ylabel("Amount Spent")
    plt.title("Restaurant Burn Rate Chart")
    plt.legend()
    plt.grid(True)
    
    # Save image in-memory
    buffer = io.BytesIO()
    plt.savefig(buffer, format='png')
    buffer.seek(0)
    encoded = base64.b64encode(buffer.read()).decode('utf-8')
    buffer.close()
    
    return f"![Burn Chart](data:image/png;base64,{encoded})"


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load data
df = pd.read_csv("restaurant_expenses.csv")
df['Amount'] = df['Amount'].replace('[\$,]', '', regex=True).astype(float)
df['Date'] = pd.to_datetime(df['Date'])

# Filter for restaurants (case-insensitive)
restaurant_df = df[df['Category'].str.lower().str.strip() == "resturants"].copy()

# Group by Date (sum per day)
daily_spend = restaurant_df.groupby('Date')['Amount'].sum().reset_index()

# Create a full date range for the month
start_date = pd.to_datetime('2025-07-01')
end_date = pd.to_datetime('2025-07-31')
all_days = pd.DataFrame({'Date': pd.date_range(start=start_date, end=end_date)})

# Merge with daily spend to fill missing days with zero
daily_spend_full = pd.merge(all_days, daily_spend, on='Date', how='left').fillna(0)

# Compute cumulative spend
daily_spend_full['Cumulative Spend'] = daily_spend_full['Amount'].cumsum()

# Budget
monthly_budget = 500
daily_spend_full['Over Budget'] = daily_spend_full['Cumulative Spend'] > monthly_budget

# Plot
plt.figure(figsize=(14, 6))
plt.plot(daily_spend_full['Date'], daily_spend_full['Cumulative Spend'], marker='o', label='Cumulative Spend', color='blue')
plt.axhline(y=monthly_budget, color='red', linestyle='--', label=f'Monthly Limit: ${monthly_budget}')

# Highlight over budget points
over_budget = daily_spend_full[daily_spend_full['Over Budget']]
plt.scatter(over_budget['Date'], over_budget['Cumulative Spend'], color='red', marker='x', s=80, label='Budget Exceeded Points')

# Formatting
plt.title("📊 Burn Rate Chart for Restaurants")
plt.xlabel("Date")
plt.ylabel("Cumulative Spend ($)")
plt.xticks(rotation=45)
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
pip install plotly

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime

def create_interactive_burn_chart(df, monthly_budget=500):
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date')

    # Set the full-month range automatically
    month_start = df['Date'].min().replace(day=1)
    month_end = pd.Timestamp(f"{month_start.year}-{month_start.month}-01") + pd.offsets.MonthEnd(0)
    df = df[(df['Date'] >= month_start) & (df['Date'] <= month_end)]

    if df.empty:
        print("No data available for the current month.")
        return {"chart_html": "<p>No data in selected month</p>"}

    df['Cumulative Spend'] = df['Amount'].cumsum()

    # Projection
    last_date = df['Date'].max()
    total_days = (last_date - month_start).days + 1
    avg_daily = df['Amount'].sum() / total_days
    projection_dates = pd.date_range(last_date + pd.Timedelta(days=1), month_end)
    projected_spend = df['Cumulative Spend'].iloc[-1] + avg_daily * range(1, len(projection_dates)+1)

    fig = go.Figure()

    # Actual spend
    fig.add_trace(go.Scatter(x=df['Date'], y=df['Cumulative Spend'],
                             mode='lines+markers', name='Actual Spend'))

    # Budget line
    fig.add_trace(go.Scatter(x=[month_start, month_end],
                             y=[monthly_budget, monthly_budget],
                             mode='lines', name=f'Monthly Limit: ${monthly_budget}',
                             line=dict(dash='dash', color='red')))

    # Projection
    fig.add_trace(go.Scatter(x=projection_dates, y=projected_spend,
                             mode='lines', name='Projected Spend',
                             line=dict(dash='dot', color='orange')))

    fig.update_layout(
        title="📊 Monthly Burn Chart for Restaurants",
        xaxis_title="Date",
        yaxis_title="Cumulative Spend ($)",
        legend_title="Legend",
        hovermode='x unified'
    )

    fig.show()
    fig_html = fig.to_html(full_html=False)
    return {"chart_html": fig_html}


In [ ]:
df = pd.read_csv("restaurant_expenses.csv")
df['Amount'] = df['Amount'].replace('[\$,]', '', regex=True).astype(float)

create_interactive_burn_chart(df, monthly_budget=500)

In [ ]:
import pandas as pd
import calendar
from datetime import datetime
from IPython.display import Markdown

def run_alert_agent(df, monthly_budget=500):
    # Clean & prep
    df = df.copy()
    df['Amount'] = df['Amount'].replace(r'[\$,]', '', regex=True).astype(float)
    df['Date'] = pd.to_datetime(df['Date'])

    # Cumulative stats
    total_spent = df['Amount'].sum()
    remaining = monthly_budget - total_spent
    spent_percent = (total_spent / monthly_budget) * 100
    status = f"⚠️ **Overspent by {round(spent_percent - 100, 1)}%**" if spent_percent > 100 else f"✅ **Under Budget ({round(100 - spent_percent, 1)}% left)**"

    # Daily limits
    daily_limit = round(monthly_budget / 31, 2)
    df['Day'] = df['Date'].dt.date
    daily_spending = df.groupby('Day')['Amount'].sum()
    days_over_limit = (daily_spending > daily_limit).sum()
    max_day = daily_spending.idxmax()
    max_day_amt = daily_spending.max()

    # Weekly stats
    df['Week'] = df['Date'].dt.to_period('W').apply(lambda r: r.start_time)
    weekly_spending = df.groupby('Week')['Amount'].sum()
    weekly_limit = round(monthly_budget / 4.33, 2)
    weeks_over_limit = (weekly_spending > weekly_limit).sum()
    max_week = weekly_spending.idxmax().strftime("%B %d")
    max_week_amt = weekly_spending.max()

    # Threshold alerts
    thresholds = [0.5, 0.75, 0.9]
    alerts = [f"🔺 **{int(t*100)}% Threshold Passed** at ${round(monthly_budget * t, 2)}"
              for t in thresholds if total_spent >= monthly_budget * t]
    alerts_md = '\n- '.join(alerts) if alerts else '✅ Spending is currently within all thresholds.'

    # Format Markdown output
    summary_md = f"""
📋 **Budget Summary**
- **Total Budget**: ${monthly_budget:.2f}  
- **Total Spent**: ${total_spent:.2f}  
- **Remaining**: ${remaining:.2f}  
- **Status**: {status}

---

📆 **Weekly Breakdown**
- **Average Weekly Limit**: ${weekly_limit:.2f}  
- **Highest Weekly Spend**: ${max_week_amt:.2f} (Week of {max_week})  
- **Weeks Over Limit**: {weeks_over_limit} of {len(weekly_spending)}

---

📅 **Daily Breakdown**
- **Average Daily Limit**: ${daily_limit:.2f}  
- **Highest Daily Spend**: ${max_day_amt:.2f} on {max_day}  
- **Days Over Limit**: {days_over_limit}

---

🚨 **Alerts & Recommendations**
- {alerts_md}
- ⚠️ **Current Spending** is {spent_percent:.1f}% of the allocated budget.
- ✅ Consider reducing future restaurant spending to stay on track.
- 🍱 Try meal prepping or alternate dining to control burn rate.

---

💡 **Next Step**
- Monitor daily spending.
- Use the burn chart to visualize when your budget was breached.
- Set automatic alerts for next month to avoid last-minute surprises.
"""

    return Markdown(summary_md)


In [ ]:
from IPython.display import display

df = pd.read_csv("restaurant_expenses.csv")
display(run_alert_agent(df, monthly_budget=500))

In [ ]:
! crewai create flow new_flow


In [ ]:
! ls -1 new_flow

In [ ]:
!crewai --version


In [ ]:
!crewai run --flow flow.yaml --inputs monthly_budget=500 expenses_file=restaurant_expenses.csv


In [ ]:
!crewai kickoff --flow flow.yaml --inputs monthly_budget=500 expenses_file=restaurant_expenses.csv


In [ ]:
pip install streamlit


In [ ]:
streamlit run budget_dashboard.py


In [ ]:
import streamlit as st
from IPython.display import Markdown

# This is your already generated output from `output.raw`
markdown_output = output.raw

# Just display it in Streamlit
st.set_page_config(page_title="Budget Alert Summary")
st.title("📋 Budget Alert Summary")
st.markdown(markdown_output)


In [ ]:
from IPython.display import HTML, display

# Convert markdown to styled HTML
html_output = f"""
<div style="background-color: #f5f7fa; padding: 20px; border-radius: 10px; font-family: sans-serif;">
    <h1 style="color: #333;">📋 Budget Alert Summary</h1>
    <div style="background: white; padding: 20px; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
        {output.raw}
    </div>
</div>
"""

display(HTML(html_output))